In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.autograd as autograd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    

class PINN_Schrodinger(nn.Module):
    # Initialize the class
    def __init__(self, x0, u0, v0, tb, X_f, layers, lb, ub):
        super(PINN_Schrodinger, self).__init__() #call __init__ from parent class 
        self.activation = nn.Tanh()
        self.iter = 0
        X0 = np.concatenate((x0, 0*x0), 1) # (x0, 0)
        X_lb = np.concatenate((0*tb + lb[0], tb), 1) # (lb[0], tb)
        X_ub = np.concatenate((0*tb + ub[0], tb), 1) # (ub[0], tb)
        
        self.lb = lb
        self.ub = ub

        self.lb = torch.from_numpy(self.lb).to(device).float()
        self.ub = torch.from_numpy(self.ub).to(device).float()
               
        self.x0 = X0[:,0:1]
        self.t0 = X0[:,1:2]

        self.x0 = torch.from_numpy(self.x0).to(device).float()
        self.t0 = torch.from_numpy(self.t0).to(device).float()

        self.x_lb = X_lb[:,0:1]
        self.t_lb = X_lb[:,1:2]
        
        self.x_lb = torch.from_numpy(self.x_lb).to(device).float()
        self.t_lb = torch.from_numpy(self.t_lb).to(device).float()

        self.x_ub = X_ub[:,0:1]
        self.t_ub = X_ub[:,1:2]

        self.x_ub = torch.from_numpy(self.x_ub).to(device).float()
        self.t_ub = torch.from_numpy(self.t_ub).to(device).float()
        
        self.x_f = X_f[:,0:1]
        self.t_f = X_f[:,1:2]
        
        self.x_f = torch.from_numpy(self.x_f).to(device).float()
        self.t_f = torch.from_numpy(self.t_f).to(device).float()

        
        self.u0 = u0
        self.v0 = v0
        
        self.u0 = torch.from_numpy(self.u0).to(device).float()
        self.v0 = torch.from_numpy(self.v0).to(device).float()

        
        
        # Initialize NNs
        self.hiddenLayers = [nn.Linear(layers[0], layers[2])] + [nn.Linear(layers[2], layers[2]) for i in range(layers[1] - 1)] + [nn.Linear(layers[2], layers[3])]
        self.linears = nn.ModuleList(self.hiddenLayers)
        self.layers = layers
        
        torch.manual_seed(1234)
        torch.cuda.manual_seed_all(1234)

        for i in range(len(layers)):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            nn.init.zeros_(self.linears[i].bias.data)

              
    def forward(self, x, t):
        
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x).to(device).float()
        if torch.is_tensor(t) != True:         
            t = torch.from_numpy(t).to(device).float()
        
        a = torch.cat((x, t), 1)
        # print(a.shape)
 
 
        for i in range(len(self.linears)-1):
            
            z = self.linears[i](a)
                        
            a = self.activation(z)
            
        a = self.linears[-1](a)
        
        return a
    
        
    def loss(self):
        criterion = nn.MSELoss()

        # loss 0
        self.x0.requires_grad = True
        self.t0.requires_grad = True
        out0 = self.forward(self.x0, self.t0)

        u0_pred, v0_pred = out0[:, [0]], out0[:, [1]]

        loss0 = criterion(u0_pred, self.u0) + criterion(v0_pred, self.v0)

        u_lb_pred, u_ub_pred, v_lb_pred, v_ub_pred, u_x_lb_pred, u_x_ub_pred, v_x_lb_pred, v_x_ub_pred = self.residual_bc()
        lossb = criterion(u_lb_pred, u_ub_pred) + criterion(v_lb_pred, v_ub_pred)

        f_u, f_v = self.residual_f()
        lossf = criterion(f_u, torch.zeros_like(f_u)) + criterion(f_v, torch.zeros_like(f_v))

        total_loss = loss0 + lossb + lossf

        return total_loss, loss0, lossb, lossf
        # lossb
    def residual_bc(self):
        self.x_lb.requires_grad = True
        self.t_lb.requires_grad = True

        outlb = self.forward(self.x_lb, self.t_lb) # lower boundary output


        u_lb_pred, v_lb_pred = outlb[:, [0]], outlb[:, [1]]

        u_x_lb_pred = autograd.grad(u_lb_pred, self.x_lb, torch.ones([self.x_lb.shape[0], 1]).to(device), retain_graph = True, create_graph = True)[0]
        v_x_lb_pred = autograd.grad(v_lb_pred, self.x_lb, torch.ones([self.x_lb.shape[0], 1]).to(device), retain_graph = True, create_graph = True)[0]


        self.x_ub.requires_grad = True
        self.t_ub.requires_grad = True

        outub = self.forward(self.x_ub, self.t_ub) # upper boundary output

        u_ub_pred, v_ub_pred = outub[:, [0]], outub[:, [1]]

        u_x_ub_pred = autograd.grad(u_ub_pred, self.x_ub, torch.ones([self.x_ub.shape[0], 1]).to(device), retain_graph = True, create_graph = True)[0]
        v_x_ub_pred = autograd.grad(v_ub_pred, self.x_ub, torch.ones([self.x_ub.shape[0], 1]).to(device), retain_graph = True, create_graph = True)[0]

        return u_lb_pred, u_ub_pred, v_lb_pred, v_ub_pred, u_x_lb_pred, u_x_ub_pred, v_x_lb_pred, v_x_ub_pred


    def residual_f(self):
        self.x_f.requires_grad = True
        self.t_f.requires_grad = True
        outf = self.forward(self.x_f, self.t_f)


        u_f_pred, v_f_pred = outf[:, [0]], outf[:, [1]]


        u_f_x_pred = autograd.grad(u_f_pred, self.x_f, torch.ones([self.x_f.shape[0], 1]).to(device), retain_graph = True, create_graph = True)[0]
        v_f_x_pred = autograd.grad(v_f_pred, self.x_f, torch.ones([self.x_f.shape[0], 1]).to(device), retain_graph = True, create_graph = True)[0]

        
        u_f_t_pred = autograd.grad(u_f_pred, self.t_f, torch.ones([self.t_f.shape[0], 1]).to(device), retain_graph = True, create_graph = True)[0]
        u_f_xx_pred = autograd.grad(u_f_x_pred, self.x_f, torch.ones([self.x_f.shape[0], 1]).to(device), retain_graph = True, create_graph = True)[0] # tf.gradients(u_x, x)[0]
        
        v_f_t_pred = autograd.grad(v_f_pred, self.t_f, torch.ones([self.t_f.shape[0], 1]).to(device), retain_graph = True, create_graph = True)[0]
        v_f_xx_pred = autograd.grad(v_f_x_pred, self.x_f, torch.ones([self.x_f.shape[0], 1]).to(device), retain_graph = True, create_graph = True)[0]
        
        f_u = u_f_t_pred + 0.5*v_f_xx_pred + (u_f_pred ** 2 + v_f_pred ** 2) * v_f_pred
        f_v = v_f_t_pred - 0.5*u_f_xx_pred - (u_f_pred ** 2 + v_f_pred ** 2) * u_f_pred

        return f_u, f_v

    

    def test(self, X_star, u_star, v_star):
        x = X_star[:, [0]]
        t = X_star[:, [1]]

        output = self.forward(x, t)
        u_pred = output[:, [0]].cpu().detach().numpy()
        v_pred = output[:, [1]].cpu().detach().numpy()
        
        h_pred = np.sqrt(u_pred**2 + v_pred**2)
        h_star = np.sqrt(u_star**2 + v_star**2)

        error_u = np.linalg.norm(u_pred - u_star, 2) / np.linalg.norm(u_star, 2)
        error_v = np.linalg.norm(v_pred - v_star, 2) / np.linalg.norm(v_star, 2)
        error_h = np.linalg.norm(h_pred - h_star, 2) / np.linalg.norm(h_star, 2)
        

        return error_u, error_v, error_h, u_pred, v_pred


class Discriminator(nn.Module):
    def __init__(self, layers):
        super(Discriminator, self).__init__()
        self.hiddenLayers = [nn.Linear(layers[0], layers[2])] + [nn.ReLU(), nn.Linear(layers[2], layers[2])] * (layers[1] - 1)+ [nn.ReLU(), nn.Linear(layers[2], layers[3])]
        self.linears = nn.ModuleList(self.hiddenLayers)

    def forward(self, input):
        # temp = torch.cat((x, t), 1)
        temp = input
        for layer in self.linears:
            temp = layer(temp)
            
        return temp

In [2]:
!pip install pyDOE

  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18170 sha256=81b370ed06c29c47e4f74bea1c0049e3b9ca2875face447b79e9810389721474
  Stored in directory: /root/.cache/pip/wheels/ce/b6/d7/c6b64746dba6433c593e471e0ac3acf4f36040456d1d160d17
Successfully built pyDOE


In [3]:
import numpy as np

import scipy.io
from pyDOE import lhs
# from plotting import newfig, savefig
import time
import torch
import torch.nn as nn
import torch.autograd as autograd

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

noise = 0.0        

# Doman bounds
lb = np.array([-5.0, 0.0])
ub = np.array([5.0, np.pi/2])

N0 = 50
N_b = 50
N_f = 20000

data = scipy.io.loadmat('/kaggle/input/shroed/NLS.mat')


t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)
Exact_h = np.sqrt(Exact_u**2 + Exact_v**2)

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact_u.T.flatten()[:,None]
v_star = Exact_v.T.flatten()[:,None]
h_star = Exact_h.T.flatten()[:,None]

###########################
np.random.seed(1234)

idx_x = np.random.choice(x.shape[0], N0, replace=False)
x0 = x[idx_x,:]
u0 = Exact_u[idx_x,0:1]
v0 = Exact_v[idx_x,0:1]

idx_t = np.random.choice(t.shape[0], N_b, replace=False)
tb = t[idx_t,:]

X_f = lb + (ub-lb)*lhs(2, N_f)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    
print(device)
torch.set_default_tensor_type(torch.FloatTensor)



# import argparse
# parser = argparse.ArgumentParser(description='Enter the parameters')
# parser.add_argument('-lr','--lr', help='learning rate', type = float, required=True)
# parser.add_argument('-pinn','--pinn', help='PINN layers and neurons', nargs="+", type=int, required=True)

# args = parser.parse_args()


# PINN set up
PINNlayers = [2, 4, 100, 2]

# customize the training
lr = 10**(-4)
PINN = PINN_Schrodinger(x0, u0, v0, tb, X_f, PINNlayers, lb, ub).to(device).float()


path = f"/kaggle/working/"




        


cpu


/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)


In [4]:
lr

0.0001

In [5]:

import pandas 
#dz = pandas.read_csv("/kaggle/input/adam_pt/pytorch/default/1/state_dict_iter_29999.pt")
dz = torch.load("/kaggle/input/adam_pytorch25/pytorch/default/1/state_dict_iter_24999.pt")
PINN.load_state_dict(dz['PINN_state_dict'])
#adam.load_state_dict(dz["optimizer_state_dict"])

<ipython-input-5-623dc53abe33>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dz = torch.load("/kaggle/input/adam_pytorch25/pytorch/default/1/state_dict_iter_24999.pt")


<All keys matched successfully>

In [6]:
# adam = torch.optim.Adam(PINN.parameters(), lr = lr)

# max_iter = 2 * 10**4
# AdamInfo = []
# print(PINN)
# print(adam)


# start_time = time.time()

# for i in range(max_iter):
#     adam.zero_grad()
#     loss, loss0, lossb, lossf = PINN.loss()
#     loss.backward()
#     adam.step()
#     # if i % 50:
#     #     print(i)
#     # if i % 10 == 0:
#     #     print([i, error_u.item(), error_v.item(), error_h.item(), loss.item(), loss0.item(), lossb.item(), lossf.item()])
#     if i % 100 == 0:
#         error_u, error_v, error_h, u_pred, v_pred = PINN.test(X_star, u_star, v_star)
#         AdamInfo.append([i, error_u.item(), error_v.item(), error_h.item(), loss.item(), loss0.item(), lossb.item(), lossf.item()])
#         # print(f"iter {i}, loss: {loss.item()}, error_u: {error_u.item()}, error_v: {error_v.item()}， error_h: {error_h.item()}")

#     if i % 5000 == 0 or i == max_iter - 1:
#             np.savetxt(path + f"/iter_{i}.csv", AdamInfo)
#             np.savetxt(path + f"/u_pred_iter_{i}.csv", u_pred)
#             np.savetxt(path + f"/v_pred_iter_{i}.csv", v_pred)
#             torch.save({
#                 "PINN_state_dict": PINN.state_dict(),
#                 "optimizer_state_dict": adam.state_dict()
#                 }, path + f"/state_dict_iter_{i}.pt")
            
            

In [7]:
#opt = [torch.optim.LBFGS(PINN.parameters(), lr=lr, max_iter=50, max_eval=50, history_size=100, line_search_fn="strong_wolfe",tolerance_change= 10**(-15)) for lr in arr]

In [8]:
LFBGS = torch.optim.LBFGS(PINN.parameters(), lr=1, max_iter=50, max_eval=50, history_size=100, line_search_fn="strong_wolfe",tolerance_change= 10**(-15))

max_iter = 2000
LFBGSInfo = []
print(PINN)
print(LFBGS)

arr = [1,0.1,0.01,0.001,0.0001]
arr2 = [50,150,300,500,5000]

start_time = time.time()
def closure():
    LFBGS.zero_grad()
    loss, loss0, lossb, lossf = PINN.loss()
    loss.backward()
    return loss
def adjust_learning_rate(optimizer, epoch):
    
    for param_group in LFBGS.param_groups:

        for i,item in enumerate(arr2):
            if epoch// item == 0:
                lr = arr[i]
                break
    
        param_group['lr'] = lr
# def adjust_opt(epoch):
#     for i,item in enumerate(arr2):
#     if epoch// item == 0:
#         LFBGS = arr[i]
#         break
#     return LFBGS
    
    
for i in range(max_iter):
    adjust_learning_rate(LFBGS,i)
    start_time = time.time()
    LFBGS.step(closure)
    
    if i % 50:
        print(i)
    loss, loss0, lossb, lossf = PINN.loss()
    print([i, loss.item()])
    if i % 10 == 0:
        error_u, error_v, error_h, u_pred, v_pred = PINN.test(X_star, u_star, v_star)
        LFBGSInfo.append([i, error_u.item(), error_v.item(), error_h.item(), loss.item(), loss0.item(), lossb.item(), lossf.item()])
        # print(f"iter {i}, loss: {loss.item()}, error_u: {error_u.item()}, error_v: {error_v.item()}， error_h: {error_h.item()}")
    if i % 5000 == 0:
        print([i, error_u.item(), error_v.item(), error_h.item(), loss.item(), loss0.item(), lossb.item(), lossf.item()])

    if i % 5000 == 0 or i == max_iter - 1:
        np.savetxt(path + f"/iter_{i}.csv", LFBGSInfo)
        np.savetxt(path + f"/u_pred_iter_{i}.csv", u_pred)
        np.savetxt(path + f"/v_pred_iter_{i}.csv", v_pred)
        torch.save({
            "PINN_state_dict": PINN.state_dict(),
            "optimizer_state_dict": LFBGS.state_dict()
            }, path + f"/state_dict_iter_{i}.pt")
    end_time = time.time()
    if ((end_time - start_time ) / 60 / 60) > 8 :
        	break



PINN_Schrodinger(
  (activation): Tanh()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=100, bias=True)
    (1-3): 3 x Linear(in_features=100, out_features=100, bias=True)
    (4): Linear(in_features=100, out_features=2, bias=True)
  )
)
LBFGS (
Parameter Group 0
    history_size: 100
    line_search_fn: strong_wolfe
    lr: 1
    max_eval: 50
    max_iter: 50
    tolerance_change: 1e-15
    tolerance_grad: 1e-07
)
[0, 7.791310781612992e-05]
[0, 0.056306168274357506, 0.09471582356561546, 0.014594507142300704, 7.791310781612992e-05, 6.44434112473391e-06, 2.4638529794174246e-07, 7.122238457668573e-05]
1
[1, 6.366532761603594e-05]
2
[2, 5.6296750699402764e-05]
3
[3, 4.757195711135864e-05]
4
[4, 4.35364454460796e-05]
5
[5, 3.8402824429795146e-05]
6
[6, 3.489945811452344e-05]
7
[7, 3.175899837515317e-05]
8
[8, 2.948851397377439e-05]
9
[9, 2.7068870622315444e-05]
10
[10, 2.5191400709445588e-05]
11
[11, 2.3605680325999856e-05]
12
[12, 2.162342207157053e-05]
13
[13, 2.052

In [9]:

(start_time - end_time  ) / 60 / 60




-0.0010079388486014472

In [10]:
max_iter = 4000
for i in range(max_iter):
    adjust_learning_rate(LFBGS,i)
    start_time = time.time()

    
    if i % 100 == 0:
        print(i)
        for param_group in LFBGS.param_groups:
            
            print(param_group['lr'] )

    
        


0
1
100
0.1
200
0.01
300
0.001
400
0.001
500
0.0001
600
0.0001
700
0.0001
800
0.0001
900
0.0001
1000
0.0001
1100
0.0001
1200
0.0001
1300
0.0001
1400
0.0001
1500
0.0001
1600
0.0001
1700
0.0001
1800
0.0001
1900
0.0001
2000
0.0001
2100
0.0001
2200
0.0001
2300
0.0001
2400
0.0001
2500
0.0001
2600
0.0001
2700
0.0001
2800
0.0001
2900
0.0001
3000
0.0001
3100
0.0001
3200
0.0001
3300
0.0001
3400
0.0001
3500
0.0001
3600
0.0001
3700
0.0001
3800
0.0001
3900
0.0001


In [11]:
current_epoch 

NameError: name 'current_epoch' is not defined

In [ ]:
40 * 10**4 / 60 / 60